In [1]:
from tutorialFunctions import *

# Tangent-space methods for uniform matrix product states
## 1. Matrix product states in the thermodynamic limit
### 1.1 Normalisation
We start by considering a uniform MPS in the thermodynamic limit, which is defined by 
$$ |\Psi(A)> = \sum_i^d \nu_L^\dagger \left[ \prod_{m\in Z} A_i \right]\nu_R |i>. $$

Here, $A_i$ are complex matrices of size $D \times D$, for every entry of the index $d$. This allows for the interpretation of the object $A$ as a three-legged tensor of dimensions $D\times d \times D$, where we will refer to $D$ as the bond dimension and $d$ as the physical dimension. With this object and the diagrammatic language of tensor networks, we can represent the state as

![image.png](img/MPSstate.png)

Thus, we initialise and represent a uniform MPS state as follows:

In [2]:
d = 3 # physical dimension
D = 5 # bond dimension
A = createMPS(D,d)

One of the most relevant objects in all our calculations will be the transfer matrix, defined as

![image.png](img/transferMatrix.png)

where we will be using the convention of ordering the legs as
1. top left
2. bottom left
3. top right
4. bottom right

In [3]:
E = createTransfer(A)

The transfer matrix can be shown to be a completely positive map, such that the leading eigenvalue is a positive number, which we conveniently rescale to unity for a proper normalization of the state in the thermodynamic limit. This means solving the (left and right) eigenvalue equation:
![image.png](img/fixedPoints.png)

Finally, we may fix the normalisation of the eigenvectors by requiring that their trace is equal to one:
![image.png](img/traceNorm.png)

In [4]:
lam1, l = leftFixedPointNaive(A)
lam2, r = rightFixedPointNaive(A)
A = A / np.sqrt(lam1)
E = createTransfer(A)

l, r = normaliseFixedPoints(l, r)

If we now require that there is no influence of the boundary on the bulk, we require the overlap of the boundary vectors with the fixed points to equal unity. In this case, we will have properly normalised the MPS:
![image.png](img/MPSNormalised.png)

### 1.2 Gauge fixing
While a single $A$ corresponds to a unique state $|\Psi(A)>$, the converse is not true, as different tensors may give rise to the same state. This is easily seen by noting that the gauge transform
![image.png](img/gaugeTransform.png)
leaves the physical state invariant. We may use this freedom in parametrization to impose canonical forms on the MPS tensor $A$.

We start by considering the left-orthonormal form $A_L$ of an MPS, which has the property that
![image.png](img/leftOrthonormal.png).

We can find the matrix $L$ that brings $A$ into this form by decomposing the fixed point $l$ as $l = L^\dagger L$, such that
![image.png](img/leftOrthonormal2.png)

In [5]:
# I can't seem to get 'naive sqrt' to work, todo

L, A_L = leftOrthonormal(A)
assert np.allclose(np.einsum('ijk,ijl->kl', A_L, np.conj(A_L)), np.eye(D)), "A_L not in left-orthonormal form"

Furthermore, there is still room for unitary gauge transformations such that we can also bring the right fixed point in diagonal form. Similarly, we can consider the right-orthonormal form $A_R$, where we have
![image.png](img/rightOrthonormal.png)

In [6]:
R, A_R = rightOrthonormal(A)
assert np.allclose(np.einsum('ijk,ljk->il', A_R, np.conj(A_R)), np.eye(D)), "A_R not in right-orthonormal form"

Finally, there is the notion of a mixed gauge for the uniform MPS by choosing a 'center site', bringing the left tensors in left-orthonormal form and the right tensors in right-orthonormal form, and redefining the center tensor as follows:
![image.png](img/mixedGauge.png)

The mixed gauge has an intuitive interpretation. Defining $C = LR$, this implements the gauge transform that maps the left-orthonormal tensor to the right-orthonormal one, hence defining the center-site tensor $A_C$:
![image.png](img/mixedGauge2.png)
The above is called the gauge condition and allows us to freely move the center tensor $A_C$ around in the MPS.

We can easily check that the above gauge fix holds for our tensors:

In [7]:
A_C = np.einsum('ij,jkl,lm->ikm', L, A, R)
C = L@R
LHS = np.einsum('ijk,kl->ijl', A_L, C)
RHS = np.einsum('ij,jkl->ikl', C, A_R)
assert np.allclose(LHS, RHS) and np.allclose(RHS, A_C), "Something went wrong in gauging the MPS"

Finally we may perform an SVD of $C = USV^\dagger$, and taking up $U$ and $V^\dagger$ in the definition of $A_L$ and $A_R$, such that we are left with a diagonal $C$ on the virtual bonds.
![image.png](img/mixedGauge3.png)

In fact, this means that we can write down a Schmidt decomposition of the state across an arbitrary bond in the chain, and the diagonal elements $C_l$ are exactly the Schmidt numbers of any bipartition of the MPS. 
![image.png](img/SchmidtDecomp.png)
Hence, we can calculate the bipartite entanglement entropy by 
$$ S = -\sum_l C_l^2 \log(C_l^2) $$

In [8]:
U, S, Vdag = svd(C)
A_L = np.einsum('ij,jkl,lm->ikm', np.conj(U).T, A_L, U)
A_R = np.einsum('ij,jkl,lm->ikm', Vdag, A_R, np.conj(Vdag).T)
C = np.diag(S)
A_C = np.einsum('ij,jkl,lm->ikm', np.conj(U).T, A_C, np.conj(Vdag).T)

LHS = np.einsum('ijk,kl->ijl', A_L, C)
RHS = np.einsum('ij,jkl->ikl', C, A_R)
assert np.allclose(LHS, RHS) and np.allclose(RHS, A_C), "Something went wrong in gauging the MPS"

entropy = -np.sum(S**2 * np.log(S))

### 1.3 Truncation of a uniform MPS
The mixed canonical form is not only useful for finding the entanglement entropy, it also allows for a way to truncate MPS states efficiently. This is done by truncating the Schmidt decomposition, such that the new MPS has a reduced bond dimension for that ond. This truncation can be shown to be optimal in the sense that the norm between the original and the truncated MPS is minimised. The truncated MPS in the mixed gauge is thus:
![image.png](img/truncMPS.png)

We mention that this is a local optimization, in the sense that it maximizes the local overlap, however not the global overlap. This would require a variational optimization of the following cost function:
$$ || ~|\Psi(A)> - |\Psi(\tilde{A})> ||^2 $$
We postpone the detailed discussion hereof until later.

In [9]:
truncate = False
if truncate:

    truncatedBondDimension = 2
    U = U[:,:truncatedBondDimension]
    Vdag = Vdag[:truncatedBondDimension,:]
    S = S[:truncatedBondDimension]

    A_L = np.einsum('ij,jkl,lm->ikm', np.conj(U).T, A_L, U)
    A_R = np.einsum('ij,jkl,lm->ikm', Vdag, A_R, np.conj(Vdag).T)
    C = np.diag(S)

# probably need to normalise again?

norm = np.einsum('ijk,ijk', A_C, np.conj(A_C))
print(norm) # not sure why but this fixes one site operator problem
A_C = A_C / np.sqrt(norm)

(0.915254012979503+0j)


### 1.4 Algorithms for finding canonical forms
One of the great things of MPS is that they provide efficient approximations of physical states, and for gapped systems they are expected to be exact in the limit $D \rightarrow \infty$. The idea is that if we increase the bond dimension enough, we can get to arbitrary precision. However, increasing the bond dimension comes at a numerical cost, as the MPS algorithms scale in $D$. It is possible to ensure that the complexity of all MPS algorithms scales as $O(D^3)$, however this means we need to be a little smarter with the previously encountered algorithms.

In [10]:
# TODO show and explain the 'smarter algorithms'

#### Note:
For the remainder of this tutorial, keep in mind that you should always aim to reduce the complexity of your algorithm to $O(D^3)$ in order to keep the computational cost tractable.

### 1.5 Computing expectation values 
Having described the states in different gauges, we want to use these to determine the expectation values of extensive operators:
$$ O = \frac{1}{Z} \sum_{n\in Z} O_n $$

If we work with properly normalized MPS, the expectation value per site of a one-body operator (such as an order parameter e.g. $X_i$) is then found by considering the following contraction:
![image.png](img/expValue.png)

In [ ]:
O = np.random.rand(d,d) + 1.0j*np.random.rand(d,d)
# convention from top to bottom

If we use the uniform gauge, we can use the fixed points of the transfer matrix to collapse everything on the left and right, such that we are left with the contraction:
![image.png](img/expValue2.png)

In [11]:
expVal = oneSiteUniform(O, A, l, r)
print(expVal)

(1.4286812128613358+1.243130303665871j) (1.4286812128613384+1.2431303036658745j)
4.440892098500626e-15


However, in the mixed gauge, we can locate the center site where the operator is acting, and then contract everything to the left and right to the identity, such that we find
![image.png](img/expValue3.png)

In [12]:
expValMix = oneSiteMixed(O, A_C)
print(expValMix)

In [ ]:
# check if exp values are the same in different gauges
diff = abs(expVal - expValMix)
assert diff < 1e-14, "different gauges give different values?"

This procedure generalises easily to operators that act on multiple sites. In particular, the Hamiltonian $h$ can be evaluated as
![image.png](img/hamExpVal.png)
#this is not correct, the contribution where Ac stands on the right is missing, i think it is, this is the Hamiltonian density, not the complete hamiltonian so there is only a single contribution. That or i dont understand what you mean?


In [13]:
# INCOMPLETE
Jx, Jy, Jz, h = 1, 1, 1, 1
H = Heisenberg(Jx, Jy, Jz, h)

# l, r = normaliseFixedPoints(l,r)
expVal = twoSiteUniform(H, A, l, r)
expValGauge = twoSiteMixed(H, A_C, A_R)
expValGauge2 = twoSiteMixed(H, A_L, A_C)

print(expVal)
print(expValGauge)
print(expValGauge2)
diff1 = abs(expVal - expValGauge)
diff2 = abs(expVal - expValGauge2)

assert diff1 < 1e-14 && diff2 < 1e-14, "different gauges give different values?"

(-1.1779834311713169-2.992397996059992e-17j) (-1.1779834311713202+5.4754041088394166e-17j)


### 1.6 Static structure factor